In [111]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time 
import xl2dict
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件


# 找到excel資料存放的位置，將所有檔案的路徑變成list

In [113]:
myxlobject= xl2dict.XlToDict()
dataDict=myxlobject.convert_sheet_to_dict(file_path=pathList[0], sheet="First Sheet")
dataDict[0]

{'個人自願提繳率(%)': '',
 '勞基法特殊身分別': '',
 '勞工保險證號': '05952202',
 '勞工保險證號檢查碼': 'F',
 '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
 '已領取社會保險給付種類': '',
 '提繳身分別': '1',
 '月實際工資': '11100',
 '格式別': '1',
 '特殊身分別': '1',
 '異動別': '4',
 '被保險人出生日期': '0880224',
 '被保險人外籍': '',
 '被保險人姓名': '吳名豪',
 '被保險人性別': '',
 '被保險人身份證號': 'A130528512',
 '雇主提繳率(%)': '6'}

# 將每個excel轉成dict，並且判斷裡面內容:
### 　　單位-保險證號
### 　　加保、退保、調保
### 　　三合一、二合一

# 讓使用者輸入:
### 　　pin碼
### 　　檔案資料夾位置

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage



try:

    pinNum="780311" #需改成人工輸入

    chrome_options = Options()
    #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
    chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
    #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
    driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
    driver.get("https://edesk.bli.gov.tw/cpa/")
    
    #找到excel資料存放的位置，將所有檔案的路徑變成list
    dataDir=r"D:\保險申報檔案" #需改成人工輸入
    #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
    # for i in os.walk(dataDir):
    #     for j in i[2]:
    #         print(i[0]+j)
    #可縮寫成這樣     
    pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
    myxlobject= xl2dict.XlToDict()
    for excelFilePath in pathList:
        #得到excel內容成dict
        dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
        #進入勞保局
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
    #     alert=driver.switch_to_alert()
    #     alert.dismiss()
        #關閉阻擋頁
        driver.find_element_by_class_name("b-close").click()
        #輸入pin碼(自然人憑證)
        pinInput=driver.find_element_by_name("pin")
        pinInput.clear()
        pinInput.send_keys(pinNum)#憑證pin碼
        #輸入保險證號
        unoInput=driver.find_element_by_name("uno")
        unoInput.clear()
        unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
        time.sleep(0.5)
        #確認進入
        driver.find_element_by_name('Submit').click()
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        passOrNot=True
        driver.find_element_by_class_name("b-close").click()
        time.sleep(0.4)
        driver.find_element_by_id("mi0,0").click()
        time.sleep(0.1)
        driver.find_element_by_id("mi0,5").click()
        time.sleep(0.2)

        if dataDict['異動別']=="4":
            driver.find_element_by_id("mi0,6").click()#加保
        elif dataDict['異動別']=="2":
            driver.find_element_by_id("mi0,7").click()#退保
        elif dataDict['異動別']=="3":
            driver.find_element_by_id("mi0,8").click()#調保
        time.sleep(0.8)

        if "健保投保單位代碼" in dataDict:
            driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
        else:
            driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

        driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
    #     time.sleep(3)
        # driver.find_element_by_id("btnSaveB").click()
        driver.find_element_by_id("mi0,53").click()
        time.sleep(1)

    driver.close()
except Exception as e:
    msg = EmailMessage()
    msg.set_content("error: "+str(e))
    msg['Subject'] = 'surcure'
    msg['From'] = "lisa"
    msg['To'] = 'andy.yuan@wowprime.com'
    with smtplib.SMTP('192.168.2.1',25) as s:
        s.send_message(msg)

In [107]:

#     {'個人自願提繳率(%)': '',
#  '勞基法特殊身分別': '',
#  '勞工保險證號': '05952202',
#  '勞工保險證號檢查碼': 'F',
#  '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
#  '已領取社會保險給付種類': '',
#  '提繳身分別': '1',
#  '月實際工資': '11100',
#  '格式別': '1',
#  '特殊身分別': '1',
#  '異動別': '4',
#  '被保險人出生日期': '0880224',
#  '被保險人外籍': '',
#  '被保險人姓名': '吳名豪',
#  '被保險人性別': '',
#  '被保險人身份證號': 'A130528512',
#  '雇主提繳率(%)': '6'}

# 學UI(tkinter)

In [16]:
from tkinter import * 
import tkinter.messagebox

class Application(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.pack()
        self.createWidgets()

    def createWidgets(self):
        self.nameInput = Entry(self)#使用者輸入
        self.nameInput.pack()
        self.alertButton = Button(self, text='Hello', command=self.hello) #按鈕
        self.alertButton.pack()

    def hello(self):
        name = self.nameInput.get() or 'world'
        messagebox.showinfo('Message', 'Hello, %s' % name)#跳出訊息視窗

In [17]:
app = Application()
app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-16-7ebc25baa391>", line 18, in hello
    messagebox.showinfo('Message', 'Hello, %s' % name)#跳出訊息視窗
NameError: name 'messagebox' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-16-7ebc25baa391>", line 18, in hello
    messagebox.showinfo('Message', 'Hello, %s' % name)#跳出訊息視窗
NameError: name 'messagebox' is not defined


In [16]:
import tkinter as tk

class SampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.entry = tk.Entry(self)
        self.button = tk.Button(self, text="Get",    command=self.on_button)
        self.button.pack()
        self.entry.pack()

    def on_button(self):
        print(self.entry.get())

app = SampleApp()
app.mainloop()

fasfs


In [19]:
from tkinter import *

master = Tk()
Label(master, text="First Name").grid(row=0)
Label(master, text="Last Name").grid(row=1)

e1 = Entry(master)
e2 = Entry(master)

e1.grid(row=0, column=1)
e2.grid(row=1, column=1)
mainloop()

In [18]:
from tkinter import *

def show_entry_fields():
    print("First Name: %s\nLast Name: %s" % (e1.get(), e2.get())) #用get方法將輸入的資料load回來

master = Tk()
Label(master, text="First Name").grid(row=0)
Label(master, text="Last Name").grid(row=1)

e1 = Entry(master)
e2 = Entry(master)

e1.grid(row=0, column=1)
e2.grid(row=1, column=1)
e1.insert(10,"Miller") #輸入預設值
e2.insert(10,"Jill") #輸入預設值

# Button(master, text='Quit', command=master.quit).grid(row=3, column=0, sticky=W, pady=4)
Button(master, text='Show', command=show_entry_fields).grid(row=3, column=1, sticky=W, pady=4)

mainloop( )

First Name: Miller
Last Name: Jill


In [27]:
from tkinter import *

def show_entry_fields():
    print("First Name: %s\nLast Name: %s" % (e1.get(), e2.get()))
    e1.delete(0,END) #印出完刪除
    e2.delete(0,END) #印出完刪除

master = Tk()
Label(master, text="First Name").grid(row=0)
Label(master, text="Last Name").grid(row=1)

e1 = Entry(master)
e2 = Entry(master)
e1.insert(10,"Miller")
e2.insert(10,"Jill")

e1.grid(row=0, column=1)
e2.grid(row=1, column=1)

Button(master, text='Quit', command=master.quit).grid(row=3,column=0, sticky=W, pady=4)
Button(master, text='Show', command=show_entry_fields).grid(row=3, column=1, sticky=W, pady=4)

mainloop( )

First Name: 1
Last Name: 3
First Name: 1241234
Last Name: 1241234
First Name: 12423dg
Last Name: 4345g24
First Name: 4
Last Name: 24
First Name: sgasg
Last Name: sgsdg


In [29]:
#!/usr/bin/python3

from tkinter import *
fields = 'Last Name', 'First Name', 'Job', 'Country'

def fetch(entries):
    for entry in entries:
        field = entry[0]
        text  = entry[1].get()
        print('%s: "%s"' % (field, text)) 

def makeform(root, fields):
    entries = []
    for field in fields:
        row = Frame(root)
        lab = Label(row, width=15, text=field, anchor='w')
        ent = Entry(row)
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT, expand=YES, fill=X)
        entries.append((field, ent))
    return entries

if __name__ == '__main__':
    root = Tk()
    ents = makeform(root, fields)
    root.bind('<Return>', (lambda event, e=ents: fetch(e)))   
    b1 = Button(root, text='Show',
      command=(lambda e=ents: fetch(e)))
    b1.pack(side=LEFT, padx=5, pady=5)
    b2 = Button(root, text='Quit', command=root.quit)
    b2.pack(side=LEFT, padx=5, pady=5)
    root.mainloop()

In [32]:
from tkinter import *

root = Tk()
T = Text(root, height=2, width=30)
T.pack()
T.insert(END, "Just a text Widget\nin two lines\n")
mainloop()

In [35]:
from tkinter import *

root = Tk()
T = Text(root, height=2, width=30)
T.pack()
quote = """HAMLET: To be, or not to be--that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune
Or to take arms against a sea of troubles
And by opposing end them. To die, to sleep--
No more--and by a sleep to say we end
The heartache, and the thousand natural shocks
That flesh is heir to. 'Tis a consummation
Devoutly to be wished."""
T.insert(END, quote)
mainloop()

In [37]:
from tkinter import *

root = Tk()
S = Scrollbar(root)
T = Text(root, height=4, width=50)
S.pack(side=RIGHT, fill=Y)
T.pack(side=LEFT, fill=Y)
S.config(command=T.yview)
T.config(yscrollcommand=S.set)
quote = """HAMLET: To be, or not to be--that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune
Or to take arms against a sea of troubles
And by opposing end them. To die, to sleep--
No more--and by a sleep to say we end
The heartache, and the thousand natural shocks  
That flesh is heir to. 'Tis a consummation
Devoutly to be wished."""
T.insert(END, quote)
mainloop(  )

In [39]:
#讓人不斷輸入資料到mydiary.txt
from tkinter import *
import tkinter as tk
class myapp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.entry=tk.Entry(self)
        self.button=tk.Button(self,text='save',command=self.on_button)
        self.button.pack()
        self.entry.pack()
    def on_button(self):
        mydiary=open('mydiary.txt','a')
        mydiary.write(self.entry.get()+'\n')
        mydiary.close()
app=myapp()
app.mainloop()

In [ ]:
#讓人輸入多行資料到mydiary.txt
from tkinter import *
import tkinter as tk
class myapp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.text=tk.Text(self)
        self.button=tk.Button(self,text='save',command=self.on_button)
        self.button1=tk.Button(self,text='quit',command=quit)
        self.button.pack()
        self.button1.pack()
        self.text.pack()
    def on_button(self):
        mydiary=open('mydiary.txt','a')
        mydiary.write(self.text.get(1.0,END)+'\n')
        mydiary.close()

app=myapp()
app.mainloop()

# 最新進度

In [82]:
from tkinter import Tk,StringVar,Label,Entry,Grid,Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage




report=[]
changDict={"4":"加保","2":"退保","3":"調保"}
web='https://edesk.bli.gov.tw/cpa/'

def doAddPopChangeAssure():
    try:
        pinNum=e2.get() #pin號碼
        chrome_options = Options()
        #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
        chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
        
        chrome_options.add_experimental_option("prefs", {
          "download.default_directory": e3.get(),
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        
        #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir=e1.get() #需改成人工輸入
        #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
        # for i in os.walk(dataDir):
        #     for j in i[2]:
        #         print(i[0]+j)
        #可縮寫成這樣     
        pathList=[i[0]+"/"+j for i in os.walk(dataDir) for j in i[2]]
        myxlobject= xl2dict.XlToDict()
        for excelFilePath in pathList:
            #得到excel內容成dict
            dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
            #進入勞保局
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
            time.sleep(0.2)
        #     alert=driver.switch_to_alert()
        #     alert.dismiss()
            #關閉阻擋頁
            driver.find_element_by_class_name("b-close").click()

            #輸入pin碼(自然人憑證)
            pinInput=driver.find_element_by_name("pin")
            pinInput.clear()
            pinInput.send_keys(pinNum)#憑證pin碼
            #輸入保險證號
            unoInput=driver.find_element_by_name("uno")
            unoInput.clear()
            unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
            time.sleep(0.2)
            #確認進入
            driver.find_element_by_name('Submit').click()
            time.sleep(2)
            try:
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                time.sleep(0.2)
                driver.find_element_by_class_name("b-close").click()
            except:
                report.append("登入失敗-異動別:{}\t證號:{}\t位置{}".
                                  format(changDict[dataDict['異動別']],
                                         dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                         excelFilePath))
                time.sleep(0.2)
                alert=driver.switch_to_alert()
                alert.accept()
                continue
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
            time.sleep(0.2)
            driver.find_element_by_id("mi0,0").click()
            time.sleep(0.2)
            driver.find_element_by_id("mi0,5").click()
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.ID,'mi0,7')))
            time.sleep(0.2)

            if dataDict['異動別']=="4":
                driver.find_element_by_id("mi0,6").click()#加保
            elif dataDict['異動別']=="2":
                driver.find_element_by_id("mi0,7").click()#退保
            elif dataDict['異動別']=="3":
                driver.find_element_by_id("mi0,8").click()#調保
            time.sleep(1)

            if "健保投保單位代碼" in dataDict:
                driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
            else:
                driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

            driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
            time.sleep(0.2)
            driver.find_element_by_id("btnSaveB").click()
            #睡到成功為止或過了15秒
            def success():
                try:
                    return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
                except:
                    return False
            nsec=0
            while (not success()):
                try:
                    if '失敗' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                        report.append("上傳失敗-異動別:{}\t證號:{}\t位置{}".
                                      format(changDict[dataDict['異動別']],
                                             dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                             excelFilePath))
                        driver.find_element_by_id("mi0,53").click()
                        continue
                except:
                    pass
                nsec+=0.5
                if nsec>15:
                    break
                time.sleep(0.5)
#                 print(1)
            if success():
                report.append("資料上傳成功-異動別:{}\t證號:{}\t位置{}".
                                  format(changDict[dataDict['異動別']],
                                         dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                         excelFilePath))
                driver.find_element_by_id("mi0,53").click()
            time.sleep(1)

        driver.quit()
        print("\n".join(report))
        msg = EmailMessage()
        msg.set_content(web+"\n"+"\n".join(report))
        msg['Subject'] = 'ReportForSuccessAndFail'
        msg['From'] = "AutoAddInsurance"
        msg['To'] = 'andy.yuan@wowprime.com'
        with smtplib.SMTP('192.168.2.1',25) as s:
            s.send_message(msg)



    except Exception as e:
        print('出錯了RRRRRRRRRRRR')
        msg = EmailMessage()
        msg.set_content("error: "+str(e)+'\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com')
        msg['Subject'] = 'ReportError'
        msg['From'] = "AutoAddInsurance"
        msg['To'] = 'andy.yuan@wowprime.com'
        with smtplib.SMTP('192.168.2.1',25) as s:
            s.send_message(msg)

def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
e3.grid(row = 1, column = 1)
Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, sticky=W, pady=4)
root.mainloop()
    
    



資料上傳成功-異動別:調保	證號:05952202F	位置D:/保險申報檔案/test/11801_調1二合一.xls
登入失敗-異動別:調保	證號:05952199DDD	位置D:/保險申報檔案/test/保證改掉sample.xls
上傳失敗-異動別:退保	證號:05952199X	位置D:/保險申報檔案/test/投保號碼改掉sample.xls
上傳失敗-異動別:加保	證號:05952199X	位置D:/保險申報檔案/test/改投保.xls
上傳失敗-異動別:調保	證號:05952199X	位置D:/保險申報檔案/test/改第二保險證號.xls


# 一步一步測試區

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage

pinNum="780311" #需改成人工輸入

chrome_options = Options()
#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://edesk.bli.gov.tw/cpa/")

#找到excel資料存放的位置，將所有檔案的路徑變成list
dataDir=r"D:\保險申報檔案" #需改成人工輸入
#遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
# for i in os.walk(dataDir):
#     for j in i[2]:
#         print(i[0]+j)
#可縮寫成這樣     
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
myxlobject= xl2dict.XlToDict()
excelFilePath=pathList[0]
#得到excel內容成dict
dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#進入勞保局
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#     alert=driver.switch_to_alert()
#     alert.dismiss()
#關閉阻擋頁
driver.find_element_by_class_name("b-close").click()
#輸入pin碼(自然人憑證)
pinInput=driver.find_element_by_name("pin")
pinInput.clear()
pinInput.send_keys(pinNum)#憑證pin碼
#輸入保險證號
unoInput=driver.find_element_by_name("uno")
unoInput.clear()

print(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']+dataDict['異動別'])
unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
time.sleep(0.5)
#確認進入
driver.find_element_by_name('Submit').click()
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
passOrNot=True
driver.find_element_by_class_name("b-close").click()
time.sleep(0.4)
driver.find_element_by_id("mi0,0").click()
time.sleep(0.1)
driver.find_element_by_id("mi0,5").click()
time.sleep(0.2)

if dataDict['異動別']=="4":
    driver.find_element_by_id("mi0,6").click()#加保
elif dataDict['異動別']=="2":
    driver.find_element_by_id("mi0,7").click()#退保
elif dataDict['異動別']=="3":
    driver.find_element_by_id("mi0,8").click()#調保
time.sleep(0.8)

if "健保投保單位代碼" in dataDict:
    driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
else:
    driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
#     time.sleep(3)
# driver.find_element_by_id("btnSaveB").click()

def success():
    try:
        return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
    except:
        return False
while not success():
    time.sleep(0.5)


05952202F3


In [81]:
pinNumList=[,,,,]
for uNo in uNoList:
    
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
    time.sleep(0.2)
    #關閉阻擋頁
    driver.find_element_by_class_name("b-close").click()
    #輸入pin碼(自然人憑證)
    pinInput=driver.find_element_by_name("pin")
    pinInput.clear()
    pinInput.send_keys(pinNum)#憑證pin碼
    #輸入保險證號
    unoInput=driver.find_element_by_name("uno")
    unoInput.clear()
    unoInput.send_keys(uNo)#"單位保險證號"
    time.sleep(0.2)
    #確認進入
    driver.find_element_by_name('Submit').click()
    time.sleep(2)
    try:
        WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        time.sleep(0.2)
        driver.find_element_by_class_name("b-close").click()
    except:
        report.append("下載資料登入失敗-證號:{}".format(uNo,))
        time.sleep(0.2)
        alert=driver.switch_to_alert()
        alert.accept()
        continue

    driver.find_element_by_id("mi0,0").click()
    time.sleep(0.1)
    driver.find_element_by_id("mi0,10").click()
    time.sleep(0.2)
    driver.find_element_by_xpath('//*[@id="main"]/form/fieldset/table/tbody/tr[7]/td[2]/input[2]').click()
    time.sleep(0.1)
    driver.find_element_by_name('btnQuery').click()
    time.sleep(0.5)

    aaa=['https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchPayList',
    'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchStopList',
    'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchAdjList']

    for i in aaa:
        driver.get(i)
        time.sleep(0.4)
        if len(driver.find_elements_by_name('meslamIdOfPrint'))>0:
            for j in driver.find_elements_by_name('meslamIdOfPrint'):
                j.click()
                driver.find_element_by_id('btnPrint2').click()
                time.sleep(0.8)
            driver.back()
            time.sleep(0.4)